# Parsing iOS decrypted Signal DB

### Imports

In [235]:
import tempfile
import sqlite3
import pandas as pd
import  plistlib
import json
import os
from PIL import Image
from io import BytesIO
import base64
import zipfile
from chat_rendering import chat_HTML, render_chat, includes, CSS

### Paths to complete

In [236]:
decrypted_db = 'path/to/signal-decrypted.sqlite'
ios_fs_zip = 'path/to/ios_fs.zip'
out_path = 'path/to/report' # to write attachments

### Constants

In [237]:
recordTypes = {56: 'baseModel',
 55: 'experienceUpgrade',
 63: 'incomingGroupsV2MessageJob',
 24: 'installedSticker',
 29: 'knownStickerPack',
 40: '_100RemoveTSRecipientsMigration',
 43: '_101ExistingUsersBlockOnIdentityChange',
 47: '_102MoveLoggingPreferenceToUserDefaults',
 42: '_103EnableVideoCalling',
 45: '_104CreateRecipientIdentities',
 44: '_105AttachmentFilePaths',
 50: '_107LegacySounds',
 48: '_108CallLoggingPreference',
 51: '_109OutgoingMessageState',
 25: 'addToContactsOfferMessage',
 7: 'addToProfileWhitelistOfferMessage',
 32: 'backupFragment',
 58: 'broadcastMediaMessageJobRecord',
 22: 'contactOffersInteraction',
 57: 'contactQuery',
 46: 'databaseMigration',
 33: 'device',
 28: 'disappearingConfigurationUpdateInfoMessage',
 39: 'disappearingMessagesConfiguration',
 61: 'incomingContactSyncJobRecord',
 60: 'incomingGroupSyncJobRecord',
 36: 'linkedDeviceReadReceipt',
 15: 'messageContentJob',
 8: 'messageDecryptJob',
 62: 'reaction',
 38: 'recipientIdentity',
 49: 'resaveCollectionDBMigration',
 52: 'sessionResetJobRecord',
 5: 'unknownContactBlockOfferMessage',
 37: 'unknownDBObject',
 54: 'unknownProtocolVersionMessage',
 41: 'userProfile',
 13: 'verificationStateChangeMessage',
 34: 'jobRecord',
 53: 'messageDecryptJobRecord',
 35: 'messageSenderJobRecord',
 30: 'signalAccount',
 31: 'signalRecipient',
 14: 'stickerPack',
 6: 'attachment',
 3: 'attachmentPointer',
 18: 'attachmentStream',
 20: 'call',
 27: 'contactThread',
 9: 'errorMessage',
 26: 'groupThread',
 19: 'incomingMessage',
 10: 'infoMessage',
 16: 'interaction',
 17: 'invalidIdentityKeyErrorMessage',
 1: 'invalidIdentityKeyReceivingErrorMessage',
 23: 'invalidIdentityKeySendingErrorMessage',
 64: 'mention',
 11: 'message',
 21: 'outgoingMessage',
 12: 'recipientReadReceipt',
 2: 'thread',
 4: 'unreadIndicatorInteraction',
 59: 'testModel'
}

mimeTypeIcon = {
    "image":"📷",
    "audio":"🎧",
    "video":"🎥",
    "animated":"🎡",
    "other":"📎",
}

In [ ]:
signal_root = '/private/var/mobile/Containers/Shared/AppGroup/DA91D9C6-9E47-4B14-8468-D347C98D599C/Attachments'

### Starting to extract

In [238]:
db = sqlite3.connect(decrypted_db)

In [317]:
threads = pd.read_sql_query("""
SELECT 
    t.contactPhoneNumber, 
    t.uniqueId as uniqueId_T, 
    t.lastInteractionRowId, 
    t.contactUUID,
    i.uniqueThreadId,
    i.attachmentIds,
    i.timestamp,
    i.receivedAtTimestamp,
    i.body,
    i.id,
    i.recordType,
    i.authorUUID,
    i.sender,
    i.configurationDurationSeconds,
    u.avatarUrlPath,
    u.profileName
    
FROM 
    model_TSInteraction i
    LEFT OUTER JOIN model_TSThread t
        ON i.uniqueThreadId = t.uniqueId
    LEFT OUTER JOIN model_OWSUserProfile u
        on t.contactPhoneNumber = u.recipientPhoneNumber
    
""", db)

attachments = pd.read_sql_query("""
SELECT
    uniqueId,
    caption,
    contentType,
    sourceFilename,
    localRelativeFilePath,
    isValidImageCached OR isValidVideoCached as cached
FROM
    model_TSAttachment
""", db)

parsedAttachmentIds field contains the attachements of a message in a binary plist format.
Currrently only supporting one attachment per message

In [318]:
def parsePlist(record):
    try:
        d = plistlib.loads(record["attachmentIds"], fmt = plistlib.FMT_BINARY)
        if d:
            if d['$objects'][1]['NS.objects']:
                return d['$objects'][2]
        return None
    except:
        return None


#### Formatting the DF and extracting attachment to pass to chat_render function

In [319]:
threads["parsedAttachmentIds"] = threads.apply(lambda record: parsePlist(record),axis=1)
threads_Att = threads.merge(attachments, left_on="parsedAttachmentIds", right_on="uniqueId", how="left")

threads_Att["recordType"] = threads.apply(lambda record: recordTypes[record["recordType"]],axis=1)
threads_Att["data-name"] = threads_Att.apply(lambda row: row["profileName"] if row["profileName"] is not None else row["contactPhoneNumber"], axis=1)
threads_Att["from_me"] = threads_Att.apply(lambda row: 1 if row["recordType"] == "outgoingMessage" else 0, axis=1)
threads_Att["body"] = threads_Att.apply(lambda row: row["recordType"] if row["recordType"] not in ["incomingMessage","outgoingMessage"] else row["body"], axis=1)
threads_Att["data-time"] = pd.to_datetime(threads_Att["timestamp"], unit='ms')
threads_Att = threads_Att.rename(columns={"body": "message","contentType":"content-type"})

os.makedirs(os.path.join(out_path,'att'), exist_ok=True)

def copyAttachments(rec,zip):
    if type(rec["localRelativeFilePath"]) == str:
        outfilename = os.path.join(out_path,'att',rec["localRelativeFilePath"].replace('/','_'))
        with open(outfilename,"wb") as out:
            out.write(zip.open(''.join([signal_root,rec["localRelativeFilePath"]])).read())
        return "/".join(outfilename.split('/')[3:])


with zipfile.ZipFile(ios_fs_zip) as zip:
    threads_Att["file-path"] = threads_Att.apply(lambda rec: copyAttachments(rec,zip), axis=1)


In [320]:
with open(os.path.join(out_path,'signal.html'),'w') as out:
    out.write('<html><head>')
    out.write(CSS)
    out.write('</head>')
    out.write(chat_HTML)
    out.write(includes)
    out.write(render_chat(threads_Att))
    out.write('</body></html>')